In [18]:
import pandas as pd
import numpy as np
import librosa
import soundfile as sd
import os
from glob import glob

In [19]:
clean_audio='/Users/anushkaanand/Desktop/venv/cleansound'
noisy_audio='/Users/anushkaanand/Desktop/venv/noise_8k'
final_audio='/Users/anushkaanand/Desktop/venv/finalsound'

In [20]:
clean_audio_files = glob(os.path.join(clean_audio, '**/*.wav'), recursive=True)
noise_files = glob(os.path.join(noisy_audio, '**/*.wav'), recursive=True)

print(f'Found {len(clean_audio_files)} clean speech files.')
print(f'Found {len(noise_files)} noise files.')

Found 1440 clean speech files.
Found 8732 noise files.


In [21]:
def add_noise(clean_signal, noise_signal):
    
    if len(noise_signal) > len(clean_signal):
        noise_signal = noise_signal[:len(clean_signal)]
    else:
        clean_signal = clean_signal[:len(noise_signal)]
    reduced_noise= 0.1*noise_signal
    noisy_signal = clean_signal + reduced_noise
    return noisy_signal

In [25]:
data = {
    "clean_mel_spectrogram": [],
    "mixed_mel_spectrogram": []
}
for i in range(1000):
    sr = 22050
    n_fft = 2048
    hop_length = 512
    n_mels = 10

    clean_file = np.random.choice(clean_audio_files)
    noise_file = np.random.choice(noise_files)
    clean_signal, sr = librosa.load(clean_file, sr=None)
    noise_signal, _ = librosa.load(noise_file, sr=None)
    noisy_signal = add_noise(clean_signal, noise_signal)
    clean_mel_spectrogram = librosa.feature.melspectrogram(y=clean_signal, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mixed_mel_spectrogram = librosa.feature.melspectrogram(y=noisy_signal, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    
    # Convert to log scale
    log_clean_mel_spectrogram = librosa.power_to_db(clean_mel_spectrogram)
    log_mixed_mel_spectrogram = librosa.power_to_db(mixed_mel_spectrogram)
    
    # Append the data to the dictionary
    data["clean_mel_spectrogram"].append(log_clean_mel_spectrogram.tolist())
    data["mixed_mel_spectrogram"].append(log_mixed_mel_spectrogram.tolist())
    
    output_file = os.path.join(final_audio, f'mixed_sample_{i:04d}.wav')
    sd.write(output_file, noisy_signal,sr)